## 引言

分词器是每个大语言模型必不可少的组件，但每个大语言模型的分词器几乎都不相同。如果要训练自己的分词器，可以使用huggingface的tokenizers框架，tokenizers包含以下主要组件：
1. Tokenizer: 分词器的核心组件，定义了分词的整个流程，包括标准化、预分词、模型分词、后处理等
2. Normalizers：可选，负责将文本标准化，包括unicode归一化、大写转小写、去重音等操作
3. Pre-tokenizers：负责将文本分割成更小的片段（如单词等），为模型分词做准备。常见的预分词器有按空格分词（Whitespace）、正则表达式分词（Regex）等
4. Models：是实际的分词算法，负责将文本片段转换为子词，常见的有BPE、WordPiece、Unigram等。
5. Post-Processors：负责对分词结果进行后处理，如添加特殊标记（CLS、SEP）。
6. Decoders：负责将分词结果转换回原始文本，常见的解码器有 ByteLevel、WordPiece 等。
7. Trainers：用于训练分词模型，不同的模型对应不同的训练器，如 BpeTrainer、WordPieceTrainer、UnigramTrainer 等。

在开始之前，先导入对应的包。

In [18]:
import json
import re
import os
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

## 加载语料库

我们准备好的语料库是一个jsonl文件，大概有736MB，每一行是一条json格式的文本数据，既有中文，也有英文。

In [14]:
!ls -l /data2/minigpt/dataset/tokenize/tokenizer_train.jsonl

-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 736729803 Nov  4 22:04 /data2/minigpt/dataset/tokenize/tokenizer_train.jsonl


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


定义一个函数用于从JSONL文件中读取文本数据，考虑到语料库会比较大，所以采用`yield`生成器来延迟到访问时再加载数据。

In [15]:
def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            data = json.loads(line)
            yield data['text']
            

In [20]:
data_path = '/data2/minigpt/dataset/tokenize/tokenizer_train.jsonl'
texts = read_texts_from_jsonl(data_path)
type(texts)

generator

可以看到，函数`read_texts_from_jsonl`返回的并不是真正的数据，而是一个生成器`generator`。可以通过`next`函数像访问`iterator`一样访问迭代数据。

In [21]:
next(texts)

'好的。现在请你将这个文本中的所有的逗号都替换成空格。 好的，请稍等一下，现在我会将文本中的所有逗号替换为空格。处理后文本为："这是一个句子 目的是看看是否可以正确地从这个句子中删除关键词。"。处理结果如何？'

## 训练过程

#### 模型选择
使用BPE模型来初始化Tokenizer实例。

In [5]:
tokenizer = Tokenizer(models.BPE())

BPE是一种基于子词的分词方法，例如：
- cats -> cat + s
- helpful -> help + ful
- congratulation -> con + gr + at + ulation

这种基于子词的分词方法，相比基于完整单词和基于单个字符有以下好处：
1. 子词相比于单词（可以认为多个子词的组合）数量要可控，这能避免词表过大，并且能避免生僻词带来的未知令牌问题。
2. 子词相比于字符语义性更强，像单个字符`f`是没有语义的，但子词`ful`可以表达`满的`，比较像英语里的词根词缀。

#### 预分词器选择
为tokenizer设置预分词器，预分词器有以下几类：
- Whitespace：按空格分隔，粒度为单词，适用于空格分隔的语言，例如英语。
- Regex：按自定义正则表达式分隔，适用于需要自定义复杂分词规则的场景。
- ByteLevel：按字节分割，适用于特殊字符、非英语场景（例如中文）。

由于我们主要面向中文，所以这里采用ByteLevel的pre_tokenizer。

In [6]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.pre_tokenizer

In [8]:
dir(tokenizer.pre_tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'add_prefix_space',
 'alphabet',
 'custom',
 'pre_tokenize',
 'pre_tokenize_str',
 'use_regex']

那pre_tokenizer具体对文本作了什么处理呢？可以通过下面几个例子来观察下。
1. 处理英文文本

In [9]:
tokenizer.pre_tokenizer.pre_tokenize_str("Pre-tokenize a :class:`~tokenizers.PyPreTokenizedString` in-place")

[('Pre', (0, 3)),
 ('-', (3, 4)),
 ('tokenize', (4, 12)),
 ('Ġa', (12, 14)),
 ('Ġ:', (14, 16)),
 ('class', (16, 21)),
 (':`~', (21, 24)),
 ('tokenizers', (24, 34)),
 ('.', (34, 35)),
 ('PyPreTokenizedString', (35, 55)),
 ('`', (55, 56)),
 ('Ġin', (56, 59)),
 ('-', (59, 60)),
 ('place', (60, 65))]

可以看到，pre_tokenizer将文本按照空格和特殊字符作了初步分词，空格处理成了特殊字符`Ġ`，并记录了每个词的起始和结束位置。

2. 处理中文文本。

In [10]:
zh_sentence = "在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。"
tokenizer.pre_tokenizer.pre_tokenize_str(zh_sentence)

[('åľ¨æŁ¥å¤ĦèĻļå¼Ģå¢ŀåĢ¼ç¨İä¸ĵçĶ¨åıĳç¥¨æ¡Īä»¶ä¸Ń', (0, 15)),
 ('ï¼Į', (15, 16)),
 ('å¸¸å¸¸æ¶īåıĬè¿Ľé¡¹çķĻæĬµç¨İé¢ĿåĴĮç¨İæ¬¾æįŁå¤±çļĦè®¤å®ļåĴĮå¤ĦçĲĨ', (16, 37)),
 ('ãĢĤ', (37, 38))]

中文基本也是按照特殊符号`，`和`。`进行了分词，但分词的结果是一堆不认识的字符，这些字符是如何产生的呢？

#### 预分词原理探究

预分词常常使用类似下面一样的正则表达式先对文本进行分隔。

In [11]:
import regex as re

PRETOKENIZE_REGEX = r"""(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}\p{P}]?\p{L}+|\p{N}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"""
pat = re.compile(PRETOKENIZE_REGEX)
tokens = re.findall(pat, zh_sentence)
tokens

['在查处虚开增值税专用发票案件中', '，', '常常涉及进项留抵税额和税款损失的认定和处理', '。']

其中，各部分正则表达式的作用如下：
-  (?i:'s|'t|'re|'ve|'m|'ll|'d): 匹配常见的英文缩略形式，例如：'s（is 或 has），'t（not），'re（are），'ve（have），'m（am），'ll（will），'d（would 或 had）。
-  [^\r\n\p{L}\p{N}\p{P}]?\p{L}+：匹配一个或多个 Unicode 字母`\p{L}`，这里的unicode字母包括英文、中文、拉丁等所有语言中的字母，允许前面有一个非换行符`\r\n`、非字母`\p{L}`、非数字`\p{N}`和非标点`\p{P}`的字符，相当于是匹配空格、制表符等空白字符。
- \p{N}：匹配任何 Unicode 数字字符。
- ?[^\s\p{L}\p{N}]+[\r\n]*：匹配非空白、非字母、非数字的字符，允许前面有一个空格，后面跟随换行符，相当于是匹配标点符号后面跟换行符。
- \p{L}：匹配任何 Unicode 字母字符，包括拉丁字母、希腊字母、汉字等所有语言中的字母。
- \p{N}：匹配任何 Unicode 数字字符，涵盖阿拉伯数字、罗马数字等所有形式的数字字符。
- \p{P}：匹配任何 Unicode 标点字符，涵盖句号、逗号、引号、括号等所有形式、所有语言中的标点符号。

对于英文以外的其它语言（例如中文），需要进行utf-8编码，将字符编码为字节，目的是解决英文、中文、日文、俄文等多语言的问题，因为世界上所有语言的字符都可以用一个或多个utf-8字节的组合来表示。

In [12]:
tokens_utf8 = [token.encode("utf-8") for token in tokens]
tokens_utf8

[b'\xe5\x9c\xa8\xe6\x9f\xa5\xe5\xa4\x84\xe8\x99\x9a\xe5\xbc\x80\xe5\xa2\x9e\xe5\x80\xbc\xe7\xa8\x8e\xe4\xb8\x93\xe7\x94\xa8\xe5\x8f\x91\xe7\xa5\xa8\xe6\xa1\x88\xe4\xbb\xb6\xe4\xb8\xad',
 b'\xef\xbc\x8c',
 b'\xe5\xb8\xb8\xe5\xb8\xb8\xe6\xb6\x89\xe5\x8f\x8a\xe8\xbf\x9b\xe9\xa1\xb9\xe7\x95\x99\xe6\x8a\xb5\xe7\xa8\x8e\xe9\xa2\x9d\xe5\x92\x8c\xe7\xa8\x8e\xe6\xac\xbe\xe6\x8d\x9f\xe5\xa4\xb1\xe7\x9a\x84\xe8\xae\xa4\xe5\xae\x9a\xe5\x92\x8c\xe5\xa4\x84\xe7\x90\x86',
 b'\xe3\x80\x82']

但有个问题是：Ascii码中是会包含回车、制表、换行等控制字符的，同样utf-8编码中也会有。而我们最终构造的词表必须是可显示的文本，所以还要做一个工作是把控制字符都转换为可显示字符，为此需要制作一个unicode字节编码表，用于将单字节（256以内）都编码为可显示字符。

0-255范围内的可显示字符分为三段：
- 从 !（ASCII 33）到 ~（ASCII 126）。
- 从 ¡（Unicode 161）到 ¬（Unicode 172）。
- 从 ®（Unicode 174）到 ÿ（Unicode 255）。

这三段以外的ASCII码均无法正常显示，需要用可显示字符来填充替代。

In [13]:
def bytes_to_unicode():
    # 收集0-255范围内的可显示字符对应的数字值，ord函数用于将字符编码为数字
    bs = (
        list(range(ord("!"), ord("~") + 1)) + 
        list(range(ord("¡"), ord("¬") + 1)) + 
        list(range(ord("®"), ord("ÿ") + 1))
    )
    cs = bs[:]
    n = 0
    # 补充0-255范围内不可显示字符对应的数字，并转换为256以上可显示字符对应的数字值
    for b in range(2**8):
        if b not in bs:
            bs.append(b)
            cs.append(2**8 + n)
            n += 1
    # chr函数用于将数字转换回unicode字符，并创建一个字节值到字符值的映射表。
    cs = [chr(n) for n in cs]
    return dict(zip(bs, cs))

byte_encoder = bytes_to_unicode()
json.dumps(byte_encoder, ensure_ascii=False)

'{"33": "!", "34": "\\"", "35": "#", "36": "$", "37": "%", "38": "&", "39": "\'", "40": "(", "41": ")", "42": "*", "43": "+", "44": ",", "45": "-", "46": ".", "47": "/", "48": "0", "49": "1", "50": "2", "51": "3", "52": "4", "53": "5", "54": "6", "55": "7", "56": "8", "57": "9", "58": ":", "59": ";", "60": "<", "61": "=", "62": ">", "63": "?", "64": "@", "65": "A", "66": "B", "67": "C", "68": "D", "69": "E", "70": "F", "71": "G", "72": "H", "73": "I", "74": "J", "75": "K", "76": "L", "77": "M", "78": "N", "79": "O", "80": "P", "81": "Q", "82": "R", "83": "S", "84": "T", "85": "U", "86": "V", "87": "W", "88": "X", "89": "Y", "90": "Z", "91": "[", "92": "\\\\", "93": "]", "94": "^", "95": "_", "96": "`", "97": "a", "98": "b", "99": "c", "100": "d", "101": "e", "102": "f", "103": "g", "104": "h", "105": "i", "106": "j", "107": "k", "108": "l", "109": "m", "110": "n", "111": "o", "112": "p", "113": "q", "114": "r", "115": "s", "116": "t", "117": "u", "118": "v", "119": "w", "120": "x", "12

> 这样，每个字节值都从 Unicode 表的开头获得一个分配给它的“可见”字符。这一点非常重要，因为每个utf-8字符都是由一到多个字节组成的，将这个长度为256的编码表中的字节进行组合，理论上就能对世界上所有语言中的字符进行编码，并且还不会出现`未知`标记。

使用这个unicode字节编码表将前面utf-8编码后的文本序列进行ByteLevel级的编码。

In [14]:
tokens_unicode = ["".join(byte_encoder[b] for b in token) for token in tokens_utf8]
tokens_unicode

['åľ¨æŁ¥å¤ĦèĻļå¼Ģå¢ŀåĢ¼ç¨İä¸ĵçĶ¨åıĳç¥¨æ¡Īä»¶ä¸Ń',
 'ï¼Į',
 'å¸¸å¸¸æ¶īåıĬè¿Ľé¡¹çķĻæĬµç¨İé¢ĿåĴĮç¨İæ¬¾æįŁå¤±çļĦè®¤å®ļåĴĮå¤ĦçĲĨ',
 'ãĢĤ']

可以看到，结果与使用pre_tokenizer预分词的结果完全相同。

#### 构建训练器
BPE训练器中需要指定几个参数：
- vocab_size：训练后词表中的词条数量，BPE是一个从短词到长词的组合过程，达到词表大小后就会停止训练。
- special_tokens：特殊token，和语言模型的特殊token相同，例如开始、结束、填充标记。
- initial_alphabet：初始字符表，使用上面长度为256的unicode字节编码表作为初始字符表。

通过`pre_tokenizers.ByteLevel.alphabet()`可以获得初始字符编码表。

In [15]:
json.dumps(pre_tokenizers.ByteLevel.alphabet(), ensure_ascii=False)

'["ø", "\\\\", "ľ", "v", "ć", "¬", "ł", "°", "ġ", "ķ", "ĕ", "»", "]", "Q", "ģ", "G", "ñ", "¶", "é", "H", "9", ")", "×", "Í", "Ó", "º", "£", "~", "Ā", "s", "Ô", "2", "Ý", "í", "â", "·", "Ą", "ý", "ĭ", "²", "4", "Ù", "ĺ", "ă", "ĸ", "Ī", "z", "K", "Ĳ", "N", "Ø", "1", "n", "b", "ó", "¼", "õ", "V", "Ö", "6", "©", "ė", "O", "đ", "j", "h", "İ", "į", "¨", "¯", "Ğ", "I", "0", "Ă", "=", "ß", "Û", "Ć", "Å", "ĩ", "Ê", "B", "ª", "W", "_", "S", "Ĥ", "Ł", "q", "ë", "Ķ", "Ò", "Ĉ", "Ċ", "L", "«", "U", "#", "ļ", "Ė", "å", "´", "Î", "M", "&", "D", "¤", "ô", "ç", "Y", "R", "ð", "Ĺ", "ĳ", ">", "Ŀ", "ę", "d", "É", "à", "ŀ", "<", "Ĩ", "¹", "Ã", "/", "¸", "Ģ", "X", "ê", "u", "ğ", "m", "w", "Ì", "¢", "Æ", "C", "t", "Ļ", "ì", "ě", "Ď", "l", "ē", "Ħ", "Ñ", "3", "÷", "{", "$", "y", "Ç", "¡", "Ë", "ĉ", "ĵ", "Ĵ", "Č", "a", "T", ";", "Ń", "Ú", "f", "§", "Z", "+", "\'", "Ä", "A", "ÿ", "Ę", "Õ", "Ġ", "c", "%", "Ē", "ï", "ī", "Ĕ", "?", "(", "Đ", "Ï", "ö", "^", "P", "±", "x", ",", "i", "æ", "®", "-", "³", "î", "*", "û",

定义特殊token，分别为填充、开始、结束标记。

In [7]:
special_tokens = ["<|endoftext|>", "<|im_start|>", "<|im_end|>"]

构建训练器，词条数量设置为32000。

In [8]:
trainer = trainers.BpeTrainer(
    vocab_size=32000,
    special_tokens=special_tokens,  # 确保这三个token被包含
    show_progress=True,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

使用上面的texts生成器作为语料库，使用trainer开始训练分词器。

In [9]:
tokenizer.train_from_iterator(texts, trainer=trainer)

这个训练过程的用时长短与文本数据大小有关，我的文本数据大概900多MB, 大概需要十几分钟。

#### 保存训练结果

在保存结果之前，需要先设置相匹配的解码器，否则ASCII以外的字符可能无法正常解码。

> 上面编码阶段使用了ByteLevel的预分词器，相对应的解码阶段也需要使用ByteLevel，表示将token id转换为token后，还需要进行一次unicode字节级别的解码，才能正常显示中文等多语言字符。

In [10]:
tokenizer.decoder = decoders.ByteLevel()

将训练的分词器保存到指定目录。

In [11]:
tokenizer_dir = "/data2/minigpt/models/tokenizer_v3"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json"))
tokenizer.model.save(tokenizer_dir)

['/data2/minigpt/models/tokenizer_v3/vocab.json',
 '/data2/minigpt/models/tokenizer_v3/merges.txt']

还需要一个分词器配置文件，包括模型类型、是否使用小写字母等。

In [12]:
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": True,
    "added_tokens_decoder": {
        "0": {
            "content": "<|endoftext|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "1": {
            "content": "<|im_start|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "2": {
            "content": "<|im_end|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        }
    },
    "additional_special_tokens": [],
    "bos_token": "<|im_start|>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "<|im_end|>",
    "legacy": True,
    "model_max_length": 1000000000000000019884624838656,
    "pad_token": None,
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<|endoftext|>",
    "use_default_system_prompt": False,
    "chat_template": "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\\n' + content + '<|im_end|>\\n<|im_start|>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|im_end|>' + '\\n' }}{% endif %}{% endfor %}"
}

In [ ]:
保存分词器配置

In [13]:
with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer training completed and saved.")

Tokenizer training completed and saved.


查看磁盘上的词表文件。

In [1]:
!ls -l /data2/minigpt/models/tokenizer_v3

total 2548
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua  407951 Oct 10 21:45 merges.txt
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua    1686 Oct 10 21:45 tokenizer_config.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua 1572840 Oct 10 21:45 tokenizer.json
-rw-rw-r-- 1 xiaoguanghua xiaoguanghua  621912 Oct 10 21:45 vocab.json


- vocab.json：词汇表文件，包含词条和对应的索引。
- merges.txt: 合并表文件，定义了子词的合并规则。
- tokenizer.json: 完整的分词器文件，它包含了分词器的所有信息，包括词汇表、合并规则、特殊标记等。
- tokenizer_config.json: 分词器配置文件，包括了起始token、结束token的定义，以及提示词模板。

## 测试分词器

In [3]:
from transformers import AutoTokenizer

# 加载预训练的tokenizer
tokenizer_dir = "/data2/minigpt/models/tokenizer_v3"
tokenizer_trained = AutoTokenizer.from_pretrained(tokenizer_dir)

'了一些'

In [ ]:
英文分词。

In [8]:
text_en = "Pre-tokenize a :class:`~tokenizers.PyPreTokenizedString` in-place"
tokenized = tokenizer_trained.tokenize(text_en)
tokenized

['Pre',
 '-',
 'token',
 'ize',
 'Ġa',
 'Ġ:',
 'class',
 ':',
 '`',
 '~',
 'token',
 'izers',
 '.',
 'Py',
 'Pre',
 'T',
 'oken',
 'ized',
 'String',
 '`',
 'Ġin',
 '-',
 'place']

tokenize方法只对输入文本作了分词，返回的是一组明文的token。要想直接返回token_id，需要使用encode方法，分词的同时完成文本到数字的序列化。

In [12]:
token_ids_en = tokenizer_trained.encode(text_en)
token_ids_en

[19714,
 15,
 24535,
 1038,
 260,
 6938,
 9939,
 28,
 66,
 96,
 24535,
 11344,
 16,
 22966,
 19714,
 54,
 9071,
 1228,
 13863,
 66,
 295,
 15,
 2383]

In [13]:
tokenizer_trained.decode(token_ids_en)

'Pre-tokenize a :class:`~tokenizers.PyPreTokenizedString` in-place'

In [ ]:
可以看到，解码的结果与原始英文串完全相同。

下面测试下中文文本的序列化和反序列化。

In [6]:
text_zh = "在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。"
token_ids_zh = tokenizer_trained.encode(text_zh)
token_ids_zh

[368,
 1698,
 1319,
 4304,
 953,
 30571,
 2147,
 411,
 646,
 3917,
 6723,
 413,
 270,
 6679,
 4743,
 631,
 1467,
 3692,
 9083,
 3534,
 2676,
 315,
 3534,
 1805,
 8576,
 269,
 1374,
 627,
 12769,
 286]

In [7]:
tokenizer_trained.decode(token_ids_zh)

'在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。'

我们刚训练的分词器在中文和英文上都能正常进行的文本的序列化和反序列化操作。

**小结**：本文借助huggingface提供的tokenizers框架，以一个真实的语料库为案例，演示了分词器训练的过程，并最终得到了一个切实可用的分词器。但tokenizers框架封装的比较多，所以在训练过程中对多语言的编码和解码部分作了内部实现的剖析和讲解，如果你还对其它部分（如BPE算法）感兴趣，下面的参考内容或许能为你提供进一步的帮助。

## 参考阅读
- [手搓BPE算法](https://golfxiao.blog.csdn.net/article/details/139106621)
- [什么是tokenizer？](https://golfxiao.blog.csdn.net/article/details/138781653)
